In [13]:
import pandas as pd
from pymongo import MongoClient

In [14]:
# Set connection
client = MongoClient("mongodb://localhost:27017/")
db = client["bank"]
balances_collection = db["balances"]

# Load csv
accounts = pd.read_csv("sample/accounts.csv")
transactions = pd.read_csv("sample/transactions.csv", parse_dates=["tran_timestamp"])

In [19]:
# Create root
balances = {}

# Take users data from account.csv
for _, row in accounts.iterrows():
    acc_id = row["acct_id"] # User id
    balances[acc_id] = [{"date": row["open_dt"],
                         "balance": row["initial_deposit"]
                        }]
    if acc_id == 0: # Check
        print(str(acc_id))
        print(row["initial_deposit"])

0
96646.86


In [21]:
# Apply transactions to users' balances
transactions.sort_values("tran_timestamp", inplace=True)

for i, (_, row) in enumerate(transactions.iterrows()):
    date = row["tran_timestamp"].strftime("%Y-%m-%d")
    orig_acct = str(row["orig_acct"])
    bene_acct = str(row["bene_acct"])

    if i == 0:
        print(orig_acct, bene_acct)

    amount = row["base_amt"]
    # aml_flag = row["is_sar"]

    if orig_acct in balances:
        new_balance = balances[orig_acct][-1]["balance"] - amount
        balances[orig_acct].append({"date": date, "balance": new_balance})
    
    if bene_acct in balances:
        new_balance = balances[bene_acct][-1]["balance"] + amount
        balances[bene_acct].append({"date": date, "balance": new_balance})

982 23


In [17]:
# Convertire i dati nel formato MongoDB e salvare
for date in sorted(set(transactions["tran_timestamp"].dt.strftime("%Y-%m-%d"))):
    daily_balances = {
        "date": date,
        "balances": {user: {"balance": b[-1]["balance"]} for user, b in balances.items()}
    }
    balances_collection.insert_one(daily_balances)

print("✅ Bilanci caricati con successo in MongoDB!")

✅ Bilanci caricati con successo in MongoDB!
